In [14]:
import csv
import subprocess
import json
import requests
import lizard

# create the project URL list
url_list = []
with open('topJavaMavenProjects.csv') as csv_file:
    i = 0
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if i == 0:
            i += 1
            continue
        url_list.append(row[0])

f = open("sstubs.json")
sstubs = json.load(f)


#  create the base table (cm_pj_usr) where
# key: commit_id
# value: project_info (dic), user_info (dic), commit_info (dic),bug_num


bug_inducing_commits = {}
for item in sstubs:
    commitId = item["fixCommitParentSHA1"]
    owner = item['projectName'].split(".")[0]
    repo = item['projectName'].split(".")[1]
    if commitId in bug_inducing_commits:
        bug_inducing_commits[commitId]['count'] += 1
    else:
        bug_inducing_commits[commitId] = {
            'count': 1,
            'owner': owner,
            'repo': repo
        }

# print remaining access 
token = "xxx"
headers = {'Authorization': "Token " + token}
rate_limit_url = "https://api.github.com/rate_limit"
req = requests.get(rate_limit_url, headers=headers).json()
print(req)
print()
print(url_list[0:10])

{'resources': {'core': {'limit': 5000, 'used': 314, 'remaining': 4686, 'reset': 1617235649}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1617232326}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1617235866}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1617235866}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1617232326}, 'code_scanning_upload': {'limit': 500, 'used': 0, 'remaining': 500, 'reset': 1617235866}}, 'rate': {'limit': 5000, 'used': 314, 'remaining': 4686, 'reset': 1617235649}}

['https://github.com/spring-projects/spring-boot', 'https://github.com/iluwatar/java-design-patterns', 'https://github.com/square/retrofit', 'https://github.com/square/okhttp', 'https://github.com/zxing/zxing', 'https://github.com/libgdx/libgdx', 'https://github.com/google/guava', 'https://github.com/alibaba/dubbo', 'https://github.com/jfeinstein10/SlidingMenu', 'https://github.com/netty/netty']


In [24]:
# build the project_info table where
# key: project_name
# value: project_info
url_list = url_list[0:1]

projects = {}
users = {}
for url in url_list:
    items = url.split("/")
    owner = items[len(items)-2]
    repo = items[len(items)-1]
    
    project_api = 'https://api.github.com/repos/' + owner + '/' + repo
    contributor_api = 'https://api.github.com/repos/' + owner + '/' + repo + '/' + "contributors"
    headers = {'Authorization': "Token " + token}
    project = requests.get(project_api, headers=headers).json()
    contributors = requests.get(contributor_api, headers=headers).json()
    #print(contributors)
    #print("##")
    # get project_info and save in a dictionary
    created_at = project['created_at']
    updated_at = project['updated_at']
    size = project['size']
    stargazers_count = project['stargazers_count']
    watchers_count = project['watchers_count']
    forks_count = project['forks_count']
    network_count = project['network_count']
    subscribers_count = project['subscribers_count']
    project_info = {
        'Owner' : owner,
        'Name' : repo,
        'Created at' : created_at,
        'Updated at' : updated_at,
        'Size' : size,
        'Stargazers' : stargazers_count,
        'Watchers' : watchers_count,
        'Forks' : forks_count,
        'Network' : network_count,
        'Subscribers' : subscribers_count
    }
    key = owner + '.' + repo
    projects[key] = project_info
    
    # get user_info and save in a dictionary
    for item in contributors:
        ## get contributor_info
        #print(item)
        username = item['login']
        contributions = item['contributions']
        
        # get the user_info
        user_api = 'https://api.github.com/users/' + username
        user = requests.get(user_api, headers=headers).json()
        user_followers = user['followers']
        user_public_repos = user['public_repos']
        user_created_at = user['created_at']
        
        user_info = {
            'contributions': contributions,
            'followers': user_followers,
            'public_repos': user_public_repos,
            'created_at': user_created_at
        }
        key = username
        users[key] = user_info
    
print(projects.keys())
print(users.keys())

dict_keys(['spring-projects.spring-boot'])
dict_keys(['wilkinsona', 'snicoll', 'philwebb', 'mbhave', 'dsyer', 'izeye', 'bclozel', 'dreis2211', 'scottfrederick', 'spring-buildmaster', 'vpavic', 'cdupuis', 'eddumelendez', 'nosan', 'garyrussell', 'sdeleuze', 'igor-suhorukov', 'ayudovin', 'Buzzardo', 'kazuki43zoo', 'michael-simons', 'artembilan', 'rwinch', 'gregturn', 'odrotbohm', 'htynkn', 'spring-operator', 'jkschneider', 'mp911de', 'mbenson'])


In [ ]:
# build the user_info table where
# key: project_user_names
# value: user_info



In [47]:
# build the commit_info table where
# key: commit_id
# value: commit_info

sliced_commits = {}
commit_keys = list(bug_inducing_commits)
for i in range(10):
    key = commit_keys[i]
    sliced_commits[key] = bug_inducing_commits[key]
#print(sliced_commits)

commits = {}
for key, value in bug_inducing_commits.items(): 
    if owner + '.' + repo != "spring-projects.spring-boot":
        continue
    commit_api = 'https://api.github.com/repos/' + owner + '/' + repo + '/commits/' + key
    headers = {'Authorization': "Token " + token}
    commit = requests.get(commit_api, headers=headers).json()
    
    # get commit_info and save in a dictionary
    #author_name = commit['commit']['author']['name']
    #author_email = commit['commit']['author']['email']
    author_id = commit['author']
    if author_id is None:
        continue
    author_id = author_id['login']
    loc = commit['stats']['total']
    additions = commit['stats']['additions']
    deletions = commit['stats']['deletions']
    files_in_change = len(commit['files'])
    commit_info = {
        #'Author name': author_name,
        #'Author email': author_email,
        'author_id': author_id,
        'LOC': loc,
        'Add': additions,
        'Delete': deletions,
        'Files': files_in_change
    }
    print(commit_info)
    commits[key] = commit_info

print("finish")

{'author_id': 'snicoll', 'LOC': 15, 'Add': 13, 'Delete': 2, 'Files': 3}
{'author_id': 'ahrytsiuk', 'LOC': 15, 'Add': 15, 'Delete': 0, 'Files': 3}
{'author_id': 'snicoll', 'LOC': 229, 'Add': 168, 'Delete': 61, 'Files': 3}
{'author_id': 'mbhave', 'LOC': 97, 'Add': 93, 'Delete': 4, 'Files': 6}
{'author_id': 'snicoll', 'LOC': 24, 'Add': 19, 'Delete': 5, 'Files': 2}
{'author_id': 'mbhave', 'LOC': 1, 'Add': 1, 'Delete': 0, 'Files': 1}
{'author_id': 'wilkinsona', 'LOC': 2, 'Add': 1, 'Delete': 1, 'Files': 1}
{'author_id': 'philwebb', 'LOC': 107, 'Add': 94, 'Delete': 13, 'Files': 4}
{'author_id': 'wilkinsona', 'LOC': 94, 'Add': 66, 'Delete': 28, 'Files': 3}
{'author_id': 'philwebb', 'LOC': 1, 'Add': 0, 'Delete': 1, 'Files': 1}
{'author_id': 'snicoll', 'LOC': 2, 'Add': 1, 'Delete': 1, 'Files': 1}
{'author_id': 'snicoll', 'LOC': 2, 'Add': 1, 'Delete': 1, 'Files': 1}
{'author_id': 'snicoll', 'LOC': 12, 'Add': 8, 'Delete': 4, 'Files': 1}
{'author_id': 'snicoll', 'LOC': 2, 'Add': 1, 'Delete': 1, 'Fi

{'author_id': 'matzschmanski', 'LOC': 2, 'Add': 1, 'Delete': 1, 'Files': 1}
{'author_id': 'dsyer', 'LOC': 5, 'Add': 2, 'Delete': 3, 'Files': 1}
{'author_id': 'philwebb', 'LOC': 6, 'Add': 1, 'Delete': 5, 'Files': 1}
{'author_id': 'wilkinsona', 'LOC': 1110, 'Add': 15, 'Delete': 1095, 'Files': 13}
{'author_id': 'snicoll', 'LOC': 290, 'Add': 216, 'Delete': 74, 'Files': 7}
{'author_id': 'rwinch', 'LOC': 119, 'Add': 116, 'Delete': 3, 'Files': 5}
{'author_id': 'joshlong', 'LOC': 140, 'Add': 92, 'Delete': 48, 'Files': 6}
{'author_id': 'cdupuis', 'LOC': 2, 'Add': 0, 'Delete': 2, 'Files': 1}
{'author_id': 'philwebb', 'LOC': 145, 'Add': 77, 'Delete': 68, 'Files': 4}
{'author_id': 'dsyer', 'LOC': 6, 'Add': 3, 'Delete': 3, 'Files': 2}
{'author_id': 'philwebb', 'LOC': 199, 'Add': 117, 'Delete': 82, 'Files': 4}
finish


In [48]:
# go through the cm_pj_usr table
# use project_name to get project_info
# use pj_usr_name to get user_info
# use commits to get commit_info
# save data in format (bug_num, project_info, user_info, commit_info) into a csv


path = "./save.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["bug_num","Watchers","contributions", "LOC"]
    csv_write.writerow(csv_head)


with open('save.csv', 'w') as f:
    csv_write = csv.writer(f)
    for commit_id in bug_inducing_commits:
        
        count = bug_inducing_commits[commit_id]['count']
        project_key = bug_inducing_commits[commit_id]['owner'] + '.' + bug_inducing_commits[commit_id]['repo']
        if project_key != "spring-projects.spring-boot":
            continue
        project_info = projects[project_key]
        if commit_id not in commits.keys():
            continue 
        commit_info = commits[commit_id]
        user_key = commit_info['author_id']
        if user_key not in users.keys():
            continue
        user_info = users[user_key]

        # get the data
        bug_num = count
        Watchers = project_info['Watchers']
        contributions = user_info['contributions']
        LOC = commit_info['LOC']
        # write into csv
        data_row = [bug_num, Watchers, contributions, LOC]
        csv_write.writerow(data_row)



